In [1]:
###Leo & Ishan
###Date: 5th November 2024
print("Hello World!")

Hello World!


In [1]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np

# for graphing
from bokeh.plotting import figure, show, output_file
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
output_file("temp_trace")

import os
from playsound import playsound
import time
import multiprocessing
import random

In [22]:
scope = cw.scope()

In [23]:
scope.default_setup()

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7370129.87012987         
scope.clock.adc_freq                     changed from 0                         to 29480519.48051948        
scope.clock.extclk_monitor_enabled       changed from True                      to False                    
scope.clock.extclk_tolerance             changed from 1144409.1796875           to 13096723.705530167       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2      

In [24]:
# scope.adc.decimate = 5
scope.adc.stream_mode = True
scope.adc.timeout = 100
# scope.adc.samples = 131_070 # is this the max for buffer, anything higher needs streaming
scope.adc.bits_per_sample = 8
scope.clock.adc_mul = 1

sample_time = 4
sample_freq = 15_000_000
scope.clock.clkgen_freq = sample_freq
scope.adc.samples = int(sample_time * sample_freq)
scope.gain.gain = 0

In [25]:
print("sample rate is:", scope.clock.adc_mul * scope.clock.clkgen_freq / 1_000_000, "MHz")
print("sampling time is:", scope.adc.samples / (scope.clock.adc_mul * scope.clock.clkgen_freq), "seconds")

sample rate is: 15.0 MHz
sampling time is: 4.0 seconds


In [26]:
scope.trigger.module = 'ADC'
scope.trigger.level = 0

In [27]:
print(scope)

cwhusky Device
sn             = 50203220325531583130353234323032
fpga_buildtime = 4/11/2024, 09:41
fw_version = 
    major = 1
    minor = 5
    debug = 0
gain = 
    mode = high
    gain = 0
    db   = 15.0
adc = 
    state                    = False
    basic_mode               = rising_edge
    timeout                  = 100
    offset                   = 0
    presamples               = 0
    samples                  = 60000000
    decimate                 = 1
    trig_count               = 0
    stream_mode              = True
    test_mode                = False
    bits_per_sample          = 8
    segments                 = 1
    segment_cycles           = 0
    segment_cycle_counter_en = False
    clip_errors_disabled     = False
    lo_gain_errors_disabled  = False
    errors                   = False
clock = 
    clkgen_src             = system
    clkgen_freq            = 15000000.0
    adc_mul                = 1
    adc_freq               = 15000000.0
    freq_ctr          

In [28]:
playsound(os.getcwd() + '/audio/trump_wake_word_delay.wav', False)

In [9]:
num_captures = 25
capture_dir = "./captures/"
capture_name = "trump_wake_word_test"

def capture_trace(scope):
    scope.arm()
    scope.capture()
    outfile = capture_dir + capture_name + "_" + str(i)
    np.save(outfile, scope.get_last_trace())

for i in range(0, num_captures):
    
    capture_process = multiprocessing.Process(
        target=capture_trace, 
        args=(scope,)
    )
    
    capture_process.start()
    time.sleep(1)
    playsound(os.getcwd() + '/audio/trump_wake_word.wav')
    capture_process.join()

In [29]:
num_captures = 200
capture_dir = "./captures/"
capture_name = "trump_wake_word_15mhz_200"

if not os.path.exists(capture_dir + capture_name): 
    os.makedirs(capture_dir + capture_name ) 

for i in range(0, num_captures):
    scope.arm()
    playsound(os.getcwd() + '/audio/trump_wake_word_delay.wav', False)
    scope.capture()
    outfile = capture_dir + capture_name + "/" + str(i)
    np.save(outfile, scope.get_last_trace(True))
    time.sleep(random.uniform(1, 3))

KeyboardInterrupt: 

In [30]:
print(scope.errors)
scope.errors.clear()
print(scope.errors)

sam_errors      = False
sam_led_setting = Default
XADC errors     = False
ADC errors      = ADC clipped, 
extclk error    = False
trace errors    = False

sam_errors      = False
sam_led_setting = Default
XADC errors     = False
ADC errors      = False
extclk error    = False
trace errors    = False



In [14]:
last_trace = scope.get_last_trace()
len(last_trace)

60000000

In [15]:
last_trace

array([ 0.01171875,  0.00390625,  0.0078125 , ..., -0.5       ,
       -0.5       , -0.5       ])

In [34]:
capture_dir = "./captures/"
capture_name = "trump_wake_word_20mhz_200"
last_trace = np.load(capture_dir + capture_name + "/" + str(87) + '.npy')

In [35]:
last_trace

array([128, 128, 129, ...,   0,   0,   0], dtype=uint8)

In [33]:
# averaging
window_size = 100
num_points = len(last_trace) // window_size
print("getting", f"{num_points:,}", "number of points")
# 1 million seems to be the plotting limit

getting 600,000 number of points


In [34]:
averaged_trace = [
    np.mean(last_trace[i * window_size:(i + 1) * window_size])
    for i in range(num_points)
]
xrange_averaged = list(range(num_points))

In [35]:
# plot figure
o = figure(width=2000)
O = o.line(xrange_averaged, averaged_trace)
show(o)

In [109]:
# Delete the traces to save memory
if 'last_trace' in locals():
    del last_trace
if 'averaged_trace' in locals():
    del averaged_trace
if 'o' in locals():
    del o

In [102]:
# plotting the beginning
o = figure(width=10000)
O = o.line(list(range(0, 1_000_000)), last_trace[0:1_000_000])
show(o)

In [108]:
# Downsampling
o = figure(width=2000)

n = 100
downsampled = last_trace[::n]  # Take every nth point
print("getting", f"{len(downsampled):,}", "number of points")

getting 1,000,000 number of points


In [109]:
xrange = list(range(len(downsampled)))
O = o.line(xrange, downsampled)
show(o)

In [195]:
scope.dis()

True

In [ ]:
import numpy as np
import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf

# Generate a large NumPy array with 100_000_000 random numbers
data = last_trace

# Create the x-axis as the index
x = np.arange(len(data))

# Create a DataFrame
df = pd.DataFrame({'x': x, 'y': data})

# Define a Datashader canvas
canvas = ds.Canvas(plot_width=20000, plot_height=20000)

# Aggregate the data
agg = canvas.line(df, 'x', 'y')

# Convert the aggregation to an image
img = tf.shade(agg, cmap=["lightblue", "darkblue"], how="linear")

# Export or display the image
img.to_pil().show()  # Display as an image

KeyboardInterrupt: 